# Métodos  Clúster usados en Word2Vec y TF-IDF

Con la base de Noticias que construimos visualizaremos los diferentes clúster que se pueden generar. 

¿Qué tan conveniente es usar un CBOW frente a un BOW en la primera exploración?

**TF-IDF** es una medida numérica que expresa la importancia relativa de un término en el documento y en todo el corpus.

$$TFIDF(t,d)=TF(t,d)\times \log\left(\cfrac{N}{DF(t)}\right)$$

In [ ]:
import pandas as pd 
Noticias=pd.read_excel("Noticias.xlsx",index_col=0)
Noticias

In [ ]:
import re 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=set(stopwords.words('spanish'))
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec


In [ ]:
Noticias2=[]
try:
    for i in Noticias["contenido"]:
        Noticias2.append(re.sub( "[^A-Za-zóáéíúÁÉÍÓÚüÜñ.]+",' ',i)) 
except Exception:
    pass

In [ ]:
Noticiasinparada=[]
for i in range(len(Noticias2)):
    Noticiasinparada.append([j.lower() for j in Noticias2[i].split() if not j.lower() in stopwords])

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
model = Word2Vec(Noticiasinparada, size=100, window=10, min_count=1, workers=4)
model.save("word2vec.model")
word_vectors = np.array([model.wv[w] for w in model.wv.vocab])
true_k = 10
kmeans = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
kmeans.fit(word_vectors)

In [ ]:
model.wv.vocab["hola"].count

In [ ]:
y=kmeans.fit_predict(word_vectors)

In [ ]:
lista=list(model.wv.vocab.keys())
palabras=[lista[int(ind)]  for ind in np.where(y==0)[0]]
conteodepalabras=[int(model.wv.vocab[i].count) for i in palabras]
masfrecuentes_ind=[np.where((-np.array(conteodepalabras)).argsort()==lm)[0][0] for lm in range(15)]
masfrecuentesenelcluster=[palabras[l] for l in masfrecuentes_ind]

In [ ]:
order_centroids = kmeans.cluster_centers_
topcluster=[]
for j in range(true_k):
    lista=list(model.wv.vocab.keys())
    palabras=[lista[int(ind)]  for ind in np.where(y==j)[0]]
    conteodepalabras=[int(model.wv.vocab[i].count) for i in palabras]
    masfrecuentes_ind=[np.where((-np.array(conteodepalabras)).argsort()==lm)[0][0] for lm in range(15)]
    masfrecuentesenelcluster=[palabras[l] for l in masfrecuentes_ind]
    topcluster.append(masfrecuentesenelcluster)
    
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in range(10):
        print(" %s" % topcluster[i][ind])

In [ ]:
Noticiasinparada2=[' '.join(i) for i in Noticiasinparada]

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(Noticiasinparada2)

In [ ]:
modelTFI = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
modelTFI.fit(X)

In [ ]:
order_centroids = modelTFI.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(" %s" % terms[ind])